In [2]:
use std::fs::File;

fn main() {
    let f = File::open("hello.txt");
}
main()

()

In [3]:
fn main() {
    let f = File::open("hello.txt");

    let f = match f {
        Ok(file) => file,
        Err(error) => {
            panic!("Problem opening the file: {:?}", error)
        },
    };
}
main()

thread '<unnamed>' panicked at 'Problem opening the file: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:9:13
stack backtrace:
   0: _ZL12preoptimized
   1: _ZL12preoptimized
   2: _ZL12preoptimized
   3: _ZL12preoptimized
   4: _ZL12preoptimized
   5: _ZL12preoptimized
   6: _ZL12preoptimized
   7: _ZL12preoptimized
   8: _ZL12preoptimized
   9: evcxr::runtime::Runtime::run_loop
  10: evcxr::runtime::runtime_hook
  11: evcxr_jupyter::main
  12: std::rt::lang_start::{{closure}}
  13: std::rt::lang_start_internal
  14: main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [2]:
// 匹配不同的错误
use std::fs::File;
use std::io::ErrorKind;

fn main() {
    let f = File::open("hello.txt");

    let f = match f {
        Ok(file) => file,
        Err(error) => match error.kind() {
            ErrorKind::NotFound => match File::create("hello.txt") {
                Ok(fc) => fc,
                Err(e) => panic!("Problem creating the file: {:?}", e),
            },
            other_error => panic!("Problem opening the file: {:?}", other_error),
        },
    };
}
main()

()

In [3]:
// Result<T, E> 有很多接受闭包的方法，并采用 match 表达式实现。一个更老练的 Rustacean 可能会这么写：
fn main() {
    let f = File::open("hello.txt").unwrap_or_else(|error| {
        if error.kind() == ErrorKind::NotFound {
            File::create("hello.txt").unwrap_or_else(|error| {
                panic!("Problem creating the file: {:?}", error);
            })
        } else {
            panic!("Problem opening the file: {:?}", error);
        }
    });
}
main()

// 在处理错误时，还有很多这类方法可以消除大量嵌套的 match 表达式。

()

In [5]:
fn main() {
    let f = File::open("hello_.txt").unwrap();
}
main()

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:6:13
stack backtrace:
   0: _ZL12preoptimized
   1: _ZL12preoptimized
   2: _ZL12preoptimized
   3: _ZL12preoptimized
   4: _ZL12preoptimized
   5: _ZL12preoptimized
   6: _ZL12preoptimized
   7: _ZL12preoptimized
   8: _ZL12preoptimized
   9: _ZL12preoptimized
  10: evcxr::runtime::Runtime::run_loop
  11: evcxr::runtime::runtime_hook
  12: evcxr_jupyter::main
  13: std::rt::lang_start::{{closure}}
  14: std::rt::lang_start_internal
  15: main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [8]:
// 使用 expect 而不是 unwrap 并提供一个好的错误信息可以表明你的意图并更易于追踪 panic 的根源。
// expect 与 unwrap 的使用方式一样：返回文件句柄或调用 panic! 宏。
// expect 用来调用 panic! 的错误信息将会作为参数传递给 expect ，
// 而不像unwrap 那样使用默认的 panic! 信息
fn main() {
    let f = File::open("hello_.txt").expect("Failed to open hello.txt");
}
main()

thread '<unnamed>' panicked at 'Failed to open hello.txt: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:7:13
stack backtrace:
   0: _ZL12preoptimized
   1: _ZL12preoptimized
   2: _ZL12preoptimized
   3: _ZL12preoptimized
   4: _ZL12preoptimized
   5: _ZL12preoptimized
   6: _ZL12preoptimized
   7: _ZL12preoptimized
   8: _ZL12preoptimized
   9: _ZL12preoptimized
  10: evcxr::runtime::Runtime::run_loop
  11: evcxr::runtime::runtime_hook
  12: evcxr_jupyter::main
  13: std::rt::lang_start::{{closure}}
  14: std::rt::lang_start_internal
  15: main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [4]:
use std::io;
use std::io::Read;
use std::fs::File;

// 一个函数使用 match 将错误返回给代码调用者
fn read_username_from_file() -> Result<String, io::Error> {
    let f = File::open("hello_.txt");

    let mut f = match f {
        Ok(file) => file,
        Err(e) => return Err(e),
    };

    let mut s = String::new();

    match f.read_to_string(&mut s) {
        Ok(_) => Ok(s),
        Err(e) => Err(e),
    }
}

read_username_from_file()

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

In [7]:
fn read_username_from_file() -> Result<String, io::Error> {
    let mut f = File::open("hello.txt")?;
    let mut s = String::new();
    f.read_to_string(&mut s)?;
    Ok(s)
}
read_username_from_file()

Ok("")

In [6]:
// ? 运算符消除了大量样板代码并使得函数的实现更简单。
// 我们甚至可以在 ? 之后直接使用链式方法调用来进一步缩短代码
fn read_username_from_file() -> Result<String, io::Error> {
    let mut s = String::new();

    File::open("hello_.txt")?.read_to_string(&mut s)?;

    Ok(s)
}
read_username_from_file()

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

Result 值之后的 **?** 被定义为与示例 9-6 中定义的处理 Result 值的 match 表达式有着完全相同的工作方式。如果 Result 的值是 Ok，这个表达式将会返回 Ok 中的值而程序将继续执行。如果值是 Err，Err 中的值将作为整个函数的返回值，就好像使用了 return 关键字一样，这样错误值就被传播给了调用者。

⊕ [Result 与可恢复的错误 · Rust 程序设计语言（第二版） 简体中文版](https://kaisery.gitbooks.io/trpl-zh-cn/content/ch09-02-recoverable-errors-with-result.html)



In [8]:
use std::fs;

// 将文件读取到一个字符串是相当常见的操作，所以 Rust 提供了名为 fs::read_to_string 的函数，
// 它会打开文件、新建一个 String、读取文件的内容，并将内容放入 String，接着返回它。
fn read_username_from_file() -> Result<String, io::Error> {
    fs::read_to_string("hello.txt")
}
read_username_from_file()

Ok("")

In [12]:
use std::error::Error;

// Box<dyn Error> 被称为 “trait 对象”（“trait object”），
// 可以理解 Box<dyn Error> 为使用 ? 时 main 允许返回的 “任何类型的错误”。
fn main() -> Result<(), Box<dyn Error>> {
    let f = File::open("hello_.txt")?;

    Ok(())
}
main()

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

In [14]:
use std::net::IpAddr;

// 硬编码的字符来创建一个 IpAddr 实例。可以看出 127.0.0.1 是一个有效的 IP 地址，所以这里使用 unwrap 是可以接受的。
let home: IpAddr = "127.0.0.1".parse().unwrap();
home

V4(127.0.0.1)

In [18]:
#[derive(Debug)]
pub struct Guess {
    value: i32,
}

impl Guess {
    pub fn new(value: i32) -> Guess {
        if value < 1 || value > 100 {
            panic!("Guess value must be between 1 and 100, got {}.", value);
        }

        Guess {
            value
        }
    }

    pub fn value(&self) -> i32 {
        self.value
    }
}

Guess::new(5)

Guess { value: 5 }